### https://stackoverflow.com/questions/18925181/procrustes-analysis-with-numpy

In [7]:
def procrustes(X, Y, scaling=True, reflection='best'):
    import numpy as np
    n,m = X.shape
    ny,my = Y.shape
    muX = X.mean(0)
    muY = Y.mean(0)
    X0 = X - muX
    Y0 = Y - muY
    ssX = (X0**2.).sum()
    ssY = (Y0**2.).sum()
    # centred Frobenius norm
    normX = np.sqrt(ssX)
    normY = np.sqrt(ssY)
    # scale to equal (unit) norm
    X0 /= normX
    Y0 /= normY
    if my < m:
        Y0 = np.concatenate((Y0, np.zeros(n, m-my)),0)
    # optimum rotation matrix of Y
    A = np.dot(X0.T, Y0)
    U,s,Vt = np.linalg.svd(A,full_matrices=False)
    V = Vt.T
    T = np.dot(V, U.T)
    if reflection != 'best':
        # does the current solution use a reflection?
        have_reflection = np.linalg.det(T) < 0
        # if that's not what was specified, force another reflection
        if reflection != have_reflection:
            V[:,-1] *= -1
            s[-1] *= -1
            T = np.dot(V, U.T)
    traceTA = s.sum()
    if scaling:
        # optimum scaling of Y
        b = traceTA * normX / normY
        # standarised distance between X and b*Y*T + c
        d = 1 - traceTA**2
        # transformed coords
        Z = normX*traceTA*np.dot(Y0, T) + muX
    else:
        b = 1
        d = 1 + ssY/ssX - 2 * traceTA * normY / normX
        Z = normY*np.dot(Y0, T) + muX
    # transformation matrix
    if my < m:
        T = T[:my,:]
    c = muX - b*np.dot(muY, T)
    #transformation values 
    tform = {'rotation':T, 'scale':b, 'translation':c}
    return d, Z, tform
import numpy as np
a = np.array([[1, 3], [1, 2], [1, 1], [2, 1]], 'd')
b = np.array([[4, -2], [4, -4], [4, -6], [2, -6]], 'd')
d, Z, tform = procrustes(a, b, scaling=True, reflection='best')
print(d, Z, tform)

0.0 [[1. 3.]
 [1. 2.]
 [1. 1.]
 [2. 1.]] {'rotation': array([[-1.00000000e+00,  8.87159524e-18],
       [-1.17202444e-17,  1.00000000e+00]]), 'scale': 0.5, 'translation': array([3., 4.])}


In [8]:
from scipy.spatial import procrustes
import numpy as np
a = np.array([[1, 3], [1, 2], [1, 1], [2, 1]], 'd')
b = np.array([[4, -2], [4, -4], [4, -6], [2, -6]], 'd')
mtx1, mtx2, disparity = procrustes(a, b)
print(mtx1, mtx2, disparity)

[[-0.13363062  0.6681531 ]
 [-0.13363062  0.13363062]
 [-0.13363062 -0.40089186]
 [ 0.40089186 -0.40089186]] [[-0.13363062  0.6681531 ]
 [-0.13363062  0.13363062]
 [-0.13363062 -0.40089186]
 [ 0.40089186 -0.40089186]] 1.4637067577342992e-32
